In [5]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [6]:
%pip install --upgrade pip
%pip install opencv-python mediapipe

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [24]:
def calculate_angle(a, b, c):
    """Returns the angle at point b (in degrees) for the triangle a‑b‑c."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))
    return angle


counter  = {"left": 0, "right": 0, "both": 0}
stage    = {"left": None, "right": None}   # 'down' → 'up'


UP_ANGLE    =  40
DOWN_ANGLE  = 160     


cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR → RGB → Pose → back to BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        
        try:
            lm = results.pose_landmarks.landmark

           
            ls, le, lw = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                          lm[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                          lm[mp_pose.PoseLandmark.LEFT_WRIST.value])
            angle_left = calculate_angle(
                (ls.x, ls.y), (le.x, le.y), (lw.x, lw.y))

            
            rs, re, rw = (lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                          lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                          lm[mp_pose.PoseLandmark.RIGHT_WRIST.value])
            angle_right = calculate_angle(
                (rs.x, rs.y), (re.x, re.y), (rw.x, rw.y))

           
            for side, angle in (("left", angle_left), ("right", angle_right)):
                if angle > DOWN_ANGLE:
                    stage[side] = "down"
                if angle < UP_ANGLE and stage[side] == "down":
                    stage[side] = "up"
                    counter[side] += 1

            
            if (stage["left"] == "up" and stage["right"] == "up"):
                counter["both"] += 1
                # prevent double‑counting until arms go down again
                stage["left"] = stage["right"] = "counted"

            
            for elbow, angle in ((le, angle_left), (re, angle_right)):
                e_px = tuple(np.multiply((elbow.x, elbow.y), [image.shape[1], image.shape[0]]).astype(int))
                cv2.putText(image, f"{int(angle)}",
                            e_px, cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except AttributeError:
            
            pass

       
        cv2.rectangle(image, (0, 0), (300, 110), (245, 117, 16), -1)

        # Line 1 : labels
        cv2.putText(image, "LEFT",  (10, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
        cv2.putText(image, "RIGHT", (80, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
        cv2.putText(image, "BOTH",  (170,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

        # Line 2 : rep counts
        cv2.putText(image, f"{counter['left']}",  (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
        cv2.putText(image, f"{counter['right']}", (80, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
        cv2.putText(image, f"{counter['both']}",  (170,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)

        # Pose skeleton
        mp_drawing.draw_landmarks(image,
                                  results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # Show 
        cv2.imshow("Mediapipe Feed", image)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
